# Required Libraries

#### Make sure to change Jupyter Notebook Kernel to .NET Interactive for csharp (must have Polyglot Notebooks extension on Visual Studio Code)

In [126]:
// dotnet tool install --global Microsoft.dotnet-interactive

// dotnet interactive jupyter install

// dotnet add package Neo4j.Driver --version 5.25.0

#r "nuget: Neo4j.Driver, 5.25.0"


Installed Packages Neo4j.Driver, 5.25.0

# Open Database Connection

In [127]:
using Neo4j.Driver;

public class Neo4jConnection
{
    private static IDriver driver;

    public static IDriver ConnectDatabase(string uri, string username, string password)
    {
        driver = GraphDatabase.Driver(uri, AuthTokens.Basic(username, password));
        return driver;
    }

    public static void CloseConnection()
    {
        driver?.Dispose();
    }
}


## CreateUser()

In [128]:
public async Task CreateUserNode(IDriver driver, Dictionary<string, string> userData)
{
    var query = @"
    CREATE (u:User {
        username: $username, 
        name: $name, 
        email: $email, 
        phone: $phone
    })
    ";

    // Run the query with the parameters
    var session = driver.AsyncSession();  // Open a session for Neo4j
    try
    {
        await session.RunAsync(query, new { 
            username = userData["username"], 
            name = userData["name"], 
            email = userData["email"], 
            phone = userData["phone"]
        });
        Console.WriteLine($"User {userData["username"]} created!");
    }
    finally
    {
        await session.CloseAsync();  // Ensure session is closed after query
    }
}

#### User Data

In [129]:
// User 1 Data
var user1Data = new Dictionary<string, string> {
    { "username", "brody675" },
    { "name", "Brody" },
    { "email", "brodykerr675@gmail.com" },
    { "phone", "832-600-8473" }
};

// User 2 Data
var user2Data = new Dictionary<string, string> {
    { "username", "jared123" },
    { "name", "Jared" },
    { "email", "jared123@gmail.com" },
    { "phone", "123-456-7890" }
};


## CreatePantry()

In [130]:
public async Task CreatePantryNode(IDriver driver, string username)
{
    var query = @"
    MATCH (user:User{username: $user})
    CREATE (pantry:Pantry {name: $pantryName})
    CREATE (user)-[:OWNS]->(pantry)
    ";
    var pantryName = username + "Pantry";

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { pantryName, user = username });
        Console.WriteLine($"Pantry node {pantryName} created!");
    }
    finally
    {
        await session.CloseAsync();
    }
}

## CreateCuisine()

In [131]:
public async Task CreateCuisineNode(IDriver driver, string cuisine)
{
    var query = @"CREATE (cuisine:Cuisine {name: $cuisineName})";

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { cuisineName = cuisine });
        Console.WriteLine($"Cuisine node {cuisine} created!");
    }
    finally
    {
        await session.CloseAsync();
    }
}


## CreateRecipe()

In [132]:
public async Task CreateRecipeNode(IDriver driver, string username, Dictionary<string, string> recipeData)
{
    var query = @"
    MATCH (user:User {username: $user})
    CREATE (recipe:Recipe {
        name: $recipeName, 
        title: $title, 
        description: $description
    })
    CREATE (user)-[:OWNS]->(recipe)
    ";

    var recipeName = username + recipeData["name"];

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { 
            user = username,
            recipeName, 
            title = recipeData["title"], 
            description = recipeData["description"]
        });
        Console.WriteLine($"Recipe {recipeName} created!");
    }
    finally
    {
        await session.CloseAsync();
    }
}

## GetRecipes()

In [133]:
public async Task<List<string>> GetRecipeNodes(IDriver driver, string username)
{
    var query = @"
    MATCH (user:User)-[:OWNS]->(recipe:Recipe)
    RETURN recipe.name AS recipeName
    ";

    var session = driver.AsyncSession();
    var recipeNames = new List<string>();

    try
    {
        // Run the query and fetch the results
        var result = await session.RunAsync(query, new { username });

        // Iterate through the result set and extract recipe names
        await result.ForEachAsync(record =>
        {
            // Add each recipe name to the list
            recipeNames.Add(record["recipeName"].As<string>());
        });

        Console.WriteLine($"Found {recipeNames.Count} recipes for user {username}!");
    }
    finally
    {
        // Ensure session is closed after query
        await session.CloseAsync();
    }

    // Return the list of recipe names
    return recipeNames;
}


#### Recipe Data

In [134]:
var recipe1Data = new Dictionary<string, string> {
    { "name", "BananaBread" },
    { "title", "Brody's Favorite Banana Bread" },
    { "description", "Has good moistness, will save for future" }
};

var recipe2Data = new Dictionary<string, string> {
    { "name", "GrilledCheese" },
    { "title", "Brody's Favorite Sammich" },
    { "description", "Very cheesy and crispy :D" }
};

var recipe3Data = new Dictionary<string, string> {
    { "name", "Chili" },
    { "title", "Brody's Favorite Chili" },
    { "description", "Very spicy" }
};

## CreateIngredient()

In [135]:
public async Task CreateIngredientNode(IDriver driver, string username, string ingredient)
{
    var query = @"CREATE (ingredient:Ingredient {name: $ingredientName})";

    var ingredientName = username + ingredient;

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { ingredientName });
        Console.WriteLine($"Ingredient node {ingredient} created!");
    }
    finally
    {
        await session.CloseAsync();
    }
}


## MergeIngredient()

In [136]:
public async Task ConnectIngredientNode(IDriver driver, string username, string parent, string ingredient)
{
    string query;
    if(parent.Contains("Pantry"))
    {
        query = @"
        MATCH (parent:Pantry{name:$parentName})
        MATCH (ingredient:Ingredient{name:$ingredientName})
        CREATE (parent)-[:STORES]->(ingredient)
        ";
    }
    else if(parent.Contains("ShopppingList"))
    {
        query = @"
        MATCH (parent:ShoppingList{name:$parentName})
        MATCH (ingredient:Ingredient{name:$ingredientName})
        CREATE (parent)-[:PLANS_TO_BUY]->(ingredient)
        ";
    }
    else
    {
        query = @"
        MATCH (parent:Recipe{name:$parentName})
        MATCH (ingredient:Ingredient{name:$ingredientName})
        CREATE (parent)-[:MADE_WITH]->(ingredient)
        ";
    }
    var parentName = username + parent;
    var ingredientName = username + ingredient;

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { parentName, ingredientName });
        Console.WriteLine($"Parent node {parentName} connected to {ingredient}!");
    }
    finally
    {
        await session.CloseAsync();
    }
}

#### Ingredient Data

This list would come from each recipe, shopping list, or pantry

In [137]:
var ingredients1 = new List<string> {
    "Flour", "Baking Soda", "Salt", "Brown Sugar", "Butter", "Eggs", "Bananas"
};

var ingredients2 = new List<string> {
    "White Bread", "Butter", "Cheddar Cheese"
}; 

var ingredients3 = new List<string> {
    "Beef", "Onion", "Tomato Sauce", "Kidney Beans", "Can Stewed Tomatos", "Water", "Chili Powder", "Garlic Powder", "Black Pepper", "Salt"
};

## CreateCookbook() & MergeCookbook() combined

In [138]:
public async Task CreateCookbookNode(IDriver driver, string username, string name)
{
    var query = @"
        MATCH (user:User{username: $user})
        CREATE (cookbook:Cookbook {name: $cookbookName})
        CREATE (user)-[:OWNS]->(cookbook)
    ";

    var cookbookName = username + name; 

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { cookbookName });
        Console.WriteLine($"Cookbook node {cookbookName} created!");
    }
    finally
    {
        await session.CloseAsync();
    }
}

## CreateTool()

## CreateGroup()

In [140]:
public async Task CreateGroupNode(IDriver driver, string group)
{
    var query = @"CREATE (group:Group {name: $groupName})";

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { groupName = group });
        Console.WriteLine($"Group node {group} created!");
    }
    finally
    {
        await session.CloseAsync();
    }
}


## CreateShopList()

In [141]:
public async Task CreateShoppingListNode(IDriver driver, string username)
{
    var query = @"CREATE (shoppinglist:ShoppingList {name: $shoppinglistName})";
    var shoppingListName = username + "ShoppingList";

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { shoppinglistName = shoppingListName });
        Console.WriteLine($"Shopping List node {shoppingListName} created!");
    }
    finally
    {
        await session.CloseAsync();
    }
}


## CreateMeal()

In [142]:
public async Task CreateMealNode(IDriver driver, string username, string meal)
{
    var query = @"
    CREATE (meal:Meal {
        name: $mealName,
        title: $meal
    })
    ";
    var mealName = username + meal;

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { mealName, meal });
        Console.WriteLine($"Meal node {mealName} created!");
    }
    finally
    {
        await session.CloseAsync();
    }
}


## MergeMeal()

In [143]:
public async Task ConnectMealNode(IDriver driver, string username, string meal, string recipe)
{
    var query = @"
    MATCH (meal:Meal{name:$mealName})
    MATCH (recipe:Recipe{name:$recipeName})
    CREATE (meal)-[:MADE_WITH]->(recipe)
    ";
    var mealName = username + meal;
    var recipeName = username + recipe;

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { mealName, recipeName });
        Console.WriteLine($"Meal node {mealName} connected to {recipe}!");
    }
    finally
    {
        await session.CloseAsync();
    }
}


#### Meal Data

In [144]:
// Meal 1 Data
var meal1Data = new List<string> { "GrilledCheese", "Chili" };
var meal1Title = "Grilled Cheese w/Chili";


## CreateMeatPlan()

## CreateStep()

In [146]:
public async Task CreateStepNode(IDriver driver, string username, Dictionary<string, object> stepData, string recipe)
{
    var query = @"
    MATCH (recipe:Recipe {name: $recipeName})
    CREATE (step:Step {
        description: $description,
        step_number: $stepNumber
    })
    CREATE (recipe)-[:HAS_STEP]->(step)
    ";

    var recipeName = username + recipe;

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { 
            description = stepData["description"], 
            stepNumber = stepData["step_number"], 
            recipeName 
        });
        Console.WriteLine($"Step {stepData["step_number"]} created for {recipeName}!");
    }
    finally
    {
        await session.CloseAsync();
    }
}


#### Step Data

In [147]:
// Step 1 Data
var step1Data = new Dictionary<string, object> {
    { "step_number", 1 },
    { "description", "Preheat a nonstick skillet over medium heat..." }
};

// Step 2 Data
var step2Data = new Dictionary<string, object> {
    { "step_number", 2 },
    { "description", "Cook until lightly browned on one side..." }
};


## CreateTag()

In [148]:
public async Task CreateTagNode(IDriver driver, string tag, string recipe, string username)
{
    var query = @"
    MATCH (recipe:Recipe {name: $recipeName})
    CREATE (tag:Tag {name: $tagName})
    CREATE (recipe)-[:TAGGED_WITH]->(tag)
    ";

    var recipeName = username + recipe;

    var session = driver.AsyncSession();
    try
    {
        await session.RunAsync(query, new { tagName = tag, recipeName });
        Console.WriteLine($"Tag {tag} created and connected to {recipeName}!");
    }
    finally
    {
        await session.CloseAsync();
    }
}


#### Tag Data

In [149]:
var tag = "Spicy";

# Main

In [150]:
// // Jared Server Login Info
// var URI = "bolt://zelpa.net:7687";
// var USERNAME = "neo4j";
// var PASSWORD = "55QvQu95HG";

// Brody's localhost login
var URI = "bolt://localhost:7687";
var USERNAME = "neo4j";
var PASSWORD = "recipeApp";

var driver = Neo4jConnection.ConnectDatabase(URI, USERNAME, PASSWORD);

// Creates a Single User Node
await CreateUserNode(driver, user1Data);
await CreateUserNode(driver, user2Data);

await CreatePantryNode(driver, user1Data["username"]);
await CreatePantryNode(driver, user2Data["username"]);

await CreateCuisineNode(driver, "Staple");

await CreateRecipeNode(driver, user1Data["username"], recipe1Data);
await CreateRecipeNode(driver, user1Data["username"], recipe2Data);
await CreateRecipeNode(driver, user1Data["username"], recipe3Data);

// This is currently creating duplicate ingredients it still needs to be fixed
foreach (var ingredient in ingredients1)
{
    await CreateIngredientNode(driver, user1Data["username"], ingredient);
    await ConnectIngredientNode(driver, user1Data["username"], recipe1Data["name"], ingredient);
    
}
foreach (var ingredient in ingredients2)
{
    await CreateIngredientNode(driver, user1Data["username"], ingredient);
    await ConnectIngredientNode(driver, user1Data["username"], recipe2Data["name"], ingredient);
}
foreach (var ingredient in ingredients3)
{
    await CreateIngredientNode(driver, user1Data["username"], ingredient);
    await ConnectIngredientNode(driver, user1Data["username"], recipe3Data["name"], ingredient);

}

await CreateStepNode(driver, user1Data["username"], step1Data, recipe2Data["name"]);
await CreateStepNode(driver, user1Data["username"], step2Data, recipe2Data["name"]);

await CreateTagNode(driver, user1Data["username"], recipe3Data["name"], tag);

await CreateGroupNode(driver, "Senior Project");

var recipes = await GetRecipeNodes(driver, user1Data["username"]);

foreach (var recipe in recipes)
{
    Console.WriteLine(recipe);
}

User brody675 created!
User jared123 created!
Pantry node brody675Pantry created!
Pantry node jared123Pantry created!
Cuisine node Staple created!
Recipe brody675BananaBread created!
Recipe brody675GrilledCheese created!
Recipe brody675Chili created!
Ingredient node Flour created!
Parent node brody675BananaBread connected to Flour!
Ingredient node Baking Soda created!
Parent node brody675BananaBread connected to Baking Soda!
Ingredient node Salt created!
Parent node brody675BananaBread connected to Salt!
Ingredient node Brown Sugar created!
Parent node brody675BananaBread connected to Brown Sugar!
Ingredient node Butter created!
Parent node brody675BananaBread connected to Butter!
Ingredient node Eggs created!
Parent node brody675BananaBread connected to Eggs!
Ingredient node Bananas created!
Parent node brody675BananaBread connected to Bananas!
Ingredient node White Bread created!
Parent node brody675GrilledCheese connected to White Bread!
Ingredient node Butter created!
Parent node b